# Generate lyrics

In [1]:
import csv
from keras import backend as K
from keras.models import load_model
import numpy as np
import os
import sys

/home/peter/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# hyper parameters
activations = 128
model_input = 'sweet dreams are made of these'
n_chars = 500

In [3]:
# output
output_dir = 'data/'
charset_file = '{}charset.csv'.format(output_dir)
model_dir = 'model/100000ex-128a-50b-20c-2018-09-19 19:41:58.hdf5'

In [4]:
# to use GPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# verify that a gpu is listed
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

## Functions

In [5]:
def deprocessPrediction(ix_to_char, prediction):
    index = np.argmax(prediction)
    char = ix_to_char[index]
    
    return char

In [6]:
def generateCharacterConverters(chars):
    char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
    ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
    
    return char_to_ix, ix_to_char

In [7]:
def preprocessInput(char_to_ix, input, n_chars_set):
    chars = list(input)
    n_sample_chars = len(chars)

    preprocessed_input = np.zeros((1, n_sample_chars, n_chars_set), dtype='float32')

    for ci, char in enumerate(chars):
        index = char_to_ix[char]
        preprocessed_input[0][ci][index] = 1

    return preprocessed_input

In [8]:
def sample_predictions(preds, temperature=0.5):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return probas

## Load Data

In [9]:
with open(charset_file, 'r') as csv_file:
    reader = csv.reader(csv_file, delimiter=",")
    charset = []
    for row in reader:
        charset.append(row[0])

## Generate Charset Dictionaries

In [10]:
# create dictionarys
char_to_ix, ix_to_char = generateCharacterConverters(charset)
n_charset = len(charset)

## Load Model

In [11]:
model = load_model(model_dir)

## Generate a sequence from a sequence

In [12]:
# convert input to a sequence of one-hot encoded chars
preprocessed_input = preprocessInput(char_to_ix, model_input, n_charset)

In [13]:
result = model_input

for i in range(n_chars):
    prediction = model.predict(preprocessed_input, verbose=0)[0]
    sampled_prediction = sample_predictions(prediction)
    next_char = deprocessPrediction(ix_to_char, sampled_prediction[0])
    preprocessed_input[0][:-1] = preprocessed_input[0][1:]
    preprocessed_input[0][-1] = sampled_prediction
    result += next_char

In [15]:
sys.stdout.write(result)

sweet dreams are made of these girl of a taits of your sings
i'm sing the choon
i can't say the little make me has all the same
i know that you're the same
i'll never the copponting to the chance to say so it to wanting that we're cry
the ends of you
and baby you sister
in the may made me and eastly girl
i can't could see it wish i'm so make you down that the hot so hart to say
the player sings
and when you say
and i hopy to latter by
the mind with you
if you take a groor is the rind
all the same of the chis the mance and th